In [ ]:
import cv2
import torch
from torchvision import models, transforms
import json
import time
import numpy as np
import os

model = models.detection.fasterrcnn_resnet50_fpn(weights=models.detection.FasterRCNN_ResNet50_FPN_Weights.DEFAULT)
model.eval()  # Set model to evaluation mode
def preprocess_image(frame):
    transform = transforms.Compose([transforms.ToTensor()])
    return transform(frame).unsqueeze(0)  # Add batch dimension
def generate_json(object_name, object_id, object_bbox, sub_object_name, sub_object_id, sub_object_bbox):
    detection_output = {
        "object": object_name,
        "id": object_id,
        "bbox": object_bbox,
        "subobject": {
            "object": sub_object_name,
            "id": sub_object_id,
            "bbox": sub_object_bbox
        }
    }
    return detection_output

def draw_boxes(frame, predictions, labels):
    for i, box in enumerate(predictions['boxes']):
        x1, y1, x2, y2 = map(int, box.tolist())
        label = labels.get(predictions['labels'][i].item(), "Unknown")
        confidence = predictions['scores'][i].item()
        if confidence > 0.5:  
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"{label}: {confidence:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    return frame
def save_cropped_image(frame, bbox, filename):
    x1, y1, x2, y2 = map(int, bbox)
    cropped_image = frame[y1:y2, x1:x2]
    cv2.imwrite(filename, cropped_image)

video_path = "video.mp4"
if not isinstance(video_path, int) and not os.path.exists(video_path):
    print(f"Error: File '{video_path}' not found.")
    exit()

cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

fps_list = []
labels_map = {1: 'Person', 2: 'Bicycle', 3: 'Car', 4: 'Motorcycle', 5: 'Airplane', 6: 'Bus', 7: 'Train', 8: 'Truck', 9: 'Boat', 10: 'Traffic Light'}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    start_time = time.time()
    img_tensor = preprocess_image(frame)
    with torch.no_grad():
        outputs = model(img_tensor)[0]

    frame = draw_boxes(frame, outputs, labels_map)
    frame = cv2.resize(frame, (1280, 720))

    end_time = time.time()
    fps = 1 / (end_time - start_time)
    fps_list.append(fps)
    cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow("Real-Time Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

if len(fps_list) > 0:
    average_fps = sum(fps_list) / len(fps_list)
    print(f"Average FPS: {average_fps:.2f}")
else:
    print("No frames processed. Cannot calculate average FPS.")

example_json = generate_json("Car", 1, [50, 50, 200, 150], "Tire", 1, [60, 60, 100, 100])
print(json.dumps(example_json, indent=4))


with open("detection_output.json", "w") as f:
    json.dump(example_json, f, indent=4)
